In [41]:
import sys

import pandas as pd

sys.path.insert(0, '../')
import map_modifiers

# Write UMLS to SNOMED map to disk

# Combine and reduce MetaMap outputs

In [2]:
umls_to_snomed = pd.read_csv('../data/umls_to_snomed.csv')

umls_to_snomed.head(1)

/home/mnz2108/miniconda3/envs/modifiers/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CUI,ISPREF,CODE,STR
0,C0000039,N,102735002,Dipalmitoylphosphatidylcholine


In [4]:
colnames = ['match', 'method', 'mm_match', 'string_name',
            'cui', 'semantic_abbrev', 'text', 'something',
            'mesh']

combined_df = pd.DataFrame()
for source in ['hrn', 'uog']:
    input_df = pd.read_csv(
        f'../data/metamap/inputs/{source}_input.txt', 
        sep='|', header=None, names=['match', 'string']
    )
    
    output_df = (
        pd.read_csv(
            f'../data/metamap/outputs/{source}_input.txt', 
            sep='|', header=None, names=colnames, 
        )
        .assign(
            NCT_id=lambda df: df['match'].apply(lambda s: s.split('$')[0].upper()),
            matched_synonym=lambda df: df['match'].apply(lambda s: s.split('$')[1]),
            match_start_index=lambda df: df['match'].apply(lambda s: s.split('$')[2]),
        )
        .filter(items=['NCT_id', 'matched_synonym', 'text', 'mm_match', 'match', 'cui'])
    )

    combined_df = pd.concat([
        combined_df,
        input_df
        .merge(output_df, on='match')
        .filter(items=['NCT_id', 'string', 'mm_match', 'cui'])
        .assign(source=source)
    ])

combined_df = (
    combined_df
    .merge(umls_to_snomed, left_on='cui', right_on='CUI', 
           how='left')
    .filter(items=['NCT_id', 'string', 'STR', 'source', 'mm_match', 'CODE'])
)

combined_df.to_csv('../data/metamap/combined_outputs.csv', index=False)

combined_df.head(2)

,NCT_id,string,STR,source,mm_match,CODE
0,NCT00000456,for disorders of major depression panic disord...,Obsessive compulsive disorder,hrn,1.38,71478004
1,NCT00000456,for disorders of major depression panic disord...,Obsessive compulsive disorder,hrn,1.38,191736004
